In [1]:
!pip install import_ipynb
import import_ipynb
from  crimes_weather import *
from divya_crime_plot_factcheck import *
import ipywidgets as widgets
from IPython.display import display
import time
from ipywidgets import Layout
import threading

importing Jupyter notebook from crimes_weather.ipynb
importing Jupyter notebook from divya_crime_plot_factcheck.ipynb


In [2]:
crimes = widgets.SelectMultiple(
    options=['THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT', 'OTHER OFFENSE',
    'BURGLARY', 'MOTOR VEHICLE THEFT', 'DECEPTIVE PRACTICE', 'ROBBERY', 'CRIMINAL TRESPASS',
    'WEAPONS VIOLATION', 'PROSTITUTION', 'OFFENSE INVOLVING CHILDREN', 'PUBLIC PEACE VIOLATION',
    'CRIM SEXUAL ASSAULT', 'SEX OFFENSE', 'INTERFERENCE WITH PUBLIC OFFICER', 'GAMBLING',
    'LIQUOR LAW VIOLATION', 'ARSON', 'HOMICIDE', 'KIDNAPPING', 'INTIMIDATION', 'STALKING',
    'OBSCENITY', 'CONCEALED CARRY LICENSE VIOLATION', 'PUBLIC INDECENCY', 'OTHER NARCOTIC VIOLATION',
    'HUMAN TRAFFICKING','RITUALISM', 'DOMESTIC VIOLENCE'
    ],
    value=['THEFT', 'BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT'],
    rows=20,
    description='Crimes',
    disabled=False,
    #layout=Layout(width='320px',border='solid',height='100px', top='0px', left='0px')
    layout=Layout(width='320px',border='solid',height='100px', top='0px', left='370px')
)

wards = widgets.SelectMultiple(
    options=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', 
             '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', 
             '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
             '41', '42', '43', '44', '45', '46', '47', '48', '49', '50'],
    value=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', 
             '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', 
             '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
             '41', '42', '43', '44', '45', '46', '47', '48', '49', '50'],
    rows=20,
    description='Wards',
    disabled=False,
    #layout=Layout(width='150px',border='solid',height='100px', top='-170px', left='325px')
    layout=Layout(width='320px',border='solid',height='100px', top='-60px', left='370px')
)

years = widgets.IntRangeSlider(
    value=[2015, 2019],
    min=2001,
    max=2019,
    step=1,
    description='Years',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    #layout=Layout(width='250px',border='solid', top='-274px', left='480px')
    layout=Layout(width='250px',border='solid', top='-274px', left='733px')
)

weather = widgets.Dropdown(
    options=['Temperature daily max (F)', 'Humidity daily mean', 'Mean Sea Level Pressure daily mean',
            'Precipitation daily sum', 'Total Cloud Cover daily mean', 'Sunshine Duration daily sum',
            'Wind Speed daily mean'],
    value='Total Cloud Cover daily mean',
    description='Weather:',
    disabled=False,
    layout=Layout(width='250px',border='solid', height='60px', top='-70px', left='733px')
)

start_date = widgets.DatePicker(
    description='started Date',
    disabled=False,
    layout=Layout(width='245px',height='30px',border='solid', top='-206px', left='733px')
)

end_date = widgets.DatePicker(
    description='ended Date',
    disabled=False,
    layout=Layout(width='250px',height='30px',border='solid', top='-203px', left='733px')
)

gkey = widgets.Password(
    description='api key:', 
    placeholder='Google api key',
    disabled=False,
    layout=Layout(width='250px',height='30px',border='solid', top='-200px', left='733px')
    #layout=Layout(width='250px',height='30px',border='solid', top='-297px', left='733px')
)

ptype = widgets.RadioButtons(
    options=['Weather Crime line plot', 
             'Weather Crime scatter plot',
             'Crime Count on Ward',
             'Crime Count Per Month line plot',
             'Crime Count Per Month bar plot',
             'Crime Distribution map',
             'SNS plot',
             'Fact Checks'],
    value='Weather Crime line plot',
    description='Plot',
    color='r',
    disabled=False,
    #layout=Layout(width='210px',height='100px',border='solid', top='-305px', left='750px')
    layout=Layout(width='330px',height='170px',border='solid', top='-400px', left='0px')
)

cond = threading.Condition()

def in_progress(count):

    f = widgets.IntProgress(min=0, 
                            description='Loading:',
                            bar_style='info', 
                            orientation='horizontal',
                            max=count) # instantiate the bar

    display(f) # display the bar

    i = 0
    cond.acquire()
    while i <= count:
        f.value += 3 # signal to increment the progress bar
        if cond.wait(3):
            i = count + 1
    cond.release()
    f.close()
    
#widget interactive plot
inprogress = False
def update_plot(crimes, weather, wards, years, start_date, end_date, key, ptype):
    global inprogress
    print('Enter update_plut() inprogress =', inprogress, "  ", ptype)
    
    if ptype ==  'Crime Distribution map':
        if start_date == None:
            print('Bad start_date=', start_date)
            return
        if end_date == None:
            print('Bad start_date=', start_date)
            return
        if key == '':
            print('Bad start_date=', start_date)
            return
        
    if inprogress == True:
        return
    inprogress = True
    x = threading.Thread(target=in_progress, args=(100,))
    x.start()
    if ptype == 'Weather Crime line plot':
        plot_crime_weather( df = crimes_weather_df, plot_type='line',  offense_types = crimes[:3],
                   meas_type = weather, years = years, from_widget = True)
    elif ptype ==   'Weather Crime scatter plot':
        plot_crime_weather( df = crimes_weather_df, plot_type='scatter',  offense_types = crimes[:3],
                   meas_type = weather, years = years, from_widget = True)
    elif ptype ==     'Crime Count Per Month line plot':
        plot_crime_counts(df = crimes_count_df, plot_type = 'line2', offense_types = crimes[:3], years = years)
    elif ptype ==     'Crime Count Per Month bar plot':
        plot_crime_counts(df = crimes_count_df, plot_type = 'bar2', offense_types = crimes[:3], years = years)
    elif ptype ==    'Crime Count on Ward':
        plot_crimes_wards(df = crimes_count_df, plot_type = 'bar', offense_types = crimes[:3], \
                       wards=[ int(i) for i in wards ], years = years, from_widget = True)
    elif ptype ==  'Crime Distribution map':
        get_crime_dist_map(df= crime_df, crime_types=crimes, start_date =start_date, end_date = end_date, key=key)
    elif ptype ==  'SNS plot':
        get_sns(df = crime_df, years=years, \
                crime_types = ['ASSAULT', 'BURGLARY', 'MOTOR VEHICLE THEFT', 'HOMICIDE', 'THEFT', 'CRIM SEXUAL ASSAULT'])
    elif ptype ==  'Fact Checks':
        get_fact_check1(df = crime_df, years=[2001, 2019], crime_types=['HOMICIDE'])
    else:
       print('not supported ...') 
    
    cond.acquire()
    cond.notify()
    cond.release()
    x.join()
    print('Return update_plut() inprogress =', inprogress, "   ", ptype)
    inprogress = False

In [3]:
x = threading.Thread(target=in_progress, args=(900,))
x.start()
print('strat reading ..... ', time.asctime(time.localtime(time.time())) )
crime_df = read_crime_data()
crimes_count_df = get_crimes_count(crime_df)
weather_df = read_weather_data()
crimes_weather_df = get_crime_weather_merged(crimes_count_df, weather_df)
cond.acquire()
cond.notify()
cond.release()
x.join()
print('done reading ..... ', time.asctime(time.localtime(time.time())) )

IntProgress(value=0, bar_style='info', description='Loading:', max=900)

strat reading .....  Thu Jul 11 13:45:16 2019
elapsed time= 748.1219937801361
columns= ['Date', 'Year', 'Month', 'Humidity daily mean', 'Mean Sea Level Pressure daily mean', 'Precipitation daily sum', 'Total Cloud Cover daily mean', 'Sunshine Duration daily sum', 'Wind Speed daily mean', 'Temperature daily max (F)']
done reading .....  Thu Jul 11 13:58:20 2019


In [4]:
#widget interactive plot

widgets.interactive(update_plot, crimes = crimes,  weather=weather, wards =wards, years=years,\
                  start_date = start_date, end_date = end_date,  key=gkey, ptype=ptype )

interactive(children=(SelectMultiple(description='Crimes', index=(0, 1, 2, 3, 4), layout=Layout(border='solid'…